# SIS hyperparameters
Generate an epidemic with some unknown "true" $(\lambda, \kappa)$, then try to infer them by doing a grid search and comparing the log-likelihood (negative Bethe free energy)

In [1]:
import Pkg; Pkg.activate("..")

  Activating project at `~/MatrixProductBP`


In [2]:
using MatrixProductBP, MatrixProductBP.Models
using Graphs, Plots, Printf, IndexedGraphs, Statistics, Random
using Base.Threads
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear);

In [33]:
include("../../telegram/notifications.jl")

default_tg_text (generic function with 1 method)

In [34]:
T = 10
N = 1000
k = 4
gg = random_regular_graph(N, k)
g = IndexedGraph(gg)
t = SimpleGraph(prim_mst(gg))
g = IndexedGraph(t)
@assert is_connected(g)
λ_true = 0.2
κ_true = 0.1
γ = 3/N;

In [35]:
sis_true = SIS(g, λ_true, κ_true, T; γ)
X, _ = onesample(mpbp(sis_true));

In [36]:
lambdas = 0.1:0.01:0.3
kappas = 0.01:0.01:0.2
f = fill!(lambdas*kappas', 0);

In [37]:
nobs = (T*N) ÷ 1

10000

In [ ]:
@telegram begin
    
for (i,λ) in enumerate(lambdas)
    println("## λ $i of ", length(lambdas))
    @threads for j in eachindex(kappas)
        κ = kappas[j]
        bp = mpbp(SIS(g, λ, κ, T; γ))
        rng = MersenneTwister(0)
        draw_node_observations!(bp.ϕ, X, nobs; rng)
        iterate!(bp; maxiter=100, tol=1e-3, showprogress=false, svd_trunc=TruncBond(6))
        f[i,j] = bethe_free_energy(bp)
    end
end
    
end

## λ 1 of 21
## λ 2 of 21
## λ 3 of 21
## λ 4 of 21


In [ ]:
heatmap(lambdas, kappas, exp.(-f)',title="likelihood of hyperparameters", xlabel="λ", ylabel="κ")
scatter!([λ_true], [κ_true], m=:plus, label="", c=:white, msw=4, ms=6)